# 1 Introduzione

Questa è una guida, ad uso personale, che ho scritto per allenarmi nell'utilizzo di alcune
funzionalità di Python.  Il dataset che verrà utilizzato riguarda i punteggi
 dei giocatori di scacchi registrati al FIDE dal 2000 al 2021, disponibili su https://ratings.fide.com/download.phtml.

Ho scelto questo dataset un po' per curiosità e un po' perché permette di applicare delle statistiche interessanti.
Si tratta infatti di un dataset abbastanza grande da costringerci ad ottimizzare le operazioni, ma
 non troppo da non farci perdere troppo tempo.

Il file load_data.py contiene le istruzioni necessarie per scaricare il dataset direttamente dal sito della FIDE.
Già qui abbiamo un problema, perché si tratta di una serie di file .txt formattati in modo diverso
a seconda della data di redazione.

Comunque alla fine otteniamo due dataset, uno contenente le statistiche mese per mese (inizialmente
solo alcuni mesi sono considerati, più recentemente abbiamo statistiche per ogni mese) e uno contenente
la lista di tutti i giocatori iscritti al FIDE. I giocatori sono identificati dal numero ID, presente
in entrambi i dataset.

Qui inizia questo notebook, che ripeto, ha sopratutto lo scopo di servire da cheatsheet.
Se invece siete interessati unicamente alle statistiche sugli scacchi, presentate in modo più organico,
consultate il file '''statistics.ipynb'''

Iniziamo ** col **